In [1]:
import pandas as pd
import zipcodes
import datetime
import gc
import numpy as np
import logging
today_str=str(datetime.datetime.now().date())

In [2]:
logging.basicConfig(filename='Saatva_Zip5_Plus4_'+today_str+'_run.log', level=logging.INFO)

In [3]:
EASI_1A=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_A2_CSV/ZIP4_18_DATA_A2.CSV",
                    skiprows=1,usecols=['ZIP_CODE','ZIP4','HH18','MEDHHINC','PCTHH200P'],dtype=str)

In [4]:
EASI_1A['HH18']=EASI_1A['HH18'].astype(float)
EASI_1A['MEDHHINC']=EASI_1A['MEDHHINC'].astype(float)
EASI_1A['PCTHH200P']=EASI_1A['PCTHH200P'].astype(float)
EASI_1A=EASI_1A.rename(columns={"MEDHHINC":"Median_HH_Inc"})

In [5]:
DMA_Zips=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
DMA_Zips=DMA_Zips.iloc[:,[0,2,6]]
DMA_Zips.columns=['ZIP_CODE','DMA','state']

In [6]:
DMA_29=pd.read_csv("/home/jian/Projects/Saatva/High_Income_Zips_for_29_DMAs/Seleceted_DMA_Top25_TV25.csv",dtype=str)
DMA_29=DMA_29['dma'].unique().tolist()

In [7]:
DMA_Zips_29=DMA_Zips[DMA_Zips['DMA'].isin(DMA_29)]

In [8]:
DMA_Zips_29=DMA_Zips_29.drop_duplicates()

In [9]:
DMA_Zips_29=pd.merge(DMA_Zips_29,EASI_1A,on="ZIP_CODE",how="left")

In [10]:
len(DMA_Zips_29['ZIP_CODE'].unique())

14289

# On Median Income Zip+4

In [11]:
# Meidan Income >= $ 100K 
DMA_Zips_29_High=DMA_Zips_29[DMA_Zips_29['Median_HH_Inc']>=100000]

In [12]:
unique_zip_city=pd.DataFrame()
for zip_cd in DMA_Zips_29_High['ZIP_CODE'].unique().tolist():
    if len(zipcodes.matching(zip_cd))>=1:
        city=zipcodes.matching(zip_cd)[0]['city']
        df=pd.DataFrame({"ZIP_CODE":zip_cd,"city":city},index=[zip_cd])
        unique_zip_city=unique_zip_city.append(df)
    else:
        print(zip_cd)
        logging.info(str(zip_cd) + ": no city in zipcodes packages, google for it")

86005
97003


In [13]:
unique_zip_city=unique_zip_city.append(pd.DataFrame({"ZIP_CODE":['86005','97003'],"city":["Kachina Village","Hillsboro"]},index=['86005','97003']))

In [14]:
DMA_Zips_29_High=pd.merge(DMA_Zips_29_High,unique_zip_city,on="ZIP_CODE",how="left")

In [15]:
DMA_Zips_29_High=DMA_Zips_29_High[['ZIP_CODE','ZIP4','DMA','city','state','HH18','Median_HH_Inc','PCTHH200P']]

In [16]:
DMA_Zips_29_High['HH_200K_Plus']=DMA_Zips_29_High['HH18']/100*DMA_Zips_29_High['PCTHH200P']

In [18]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/High_Income_Zips_for_29_DMAs/Saatva_Zip Plus4 Median HH Inc above 100K for 29 DMAs_JL_20180823.xlsx",engine="xlsxwriter")

In [19]:
for dma in DMA_29:
    df=DMA_Zips_29_High[DMA_Zips_29_High['DMA']==dma]
    df=df.reset_index()
    del df['index']
    if df.shape[0]>1000000:
        df=df.sort_values('Avg_HH_Inc',ascending=False)
        df=df.reset_index()
        del df['index']
        df_0=df.iloc[0:50000,]
        df_1=df.iloc[50000:,]
        df_0.to_excel(writer,dma+"_1",index=False)
        df_1.to_excel(writer,dma+"_2",index=False)
    else:
        df.to_excel(writer,dma,index=False)
    print(dma+" with rows of: "+str(df.shape[0]))

NEW YORK with rows of: 787428
LOS ANGELES with rows of: 407325
SAN FRANCISCO-OAK-SAN JOSE with rows of: 290523
WASHINGTON, DC (HAGRSTWN) with rows of: 360076
CHICAGO with rows of: 282954
BOSTON (MANCHESTER) with rows of: 291974
SEATTLE-TACOMA with rows of: 161328
PHILADELPHIA with rows of: 232198
DALLAS-FT. WORTH with rows of: 213575
DENVER with rows of: 126332
ATLANTA with rows of: 143564
PHOENIX (PRESCOTT) with rows of: 122719
TAMPA-ST. PETE (SARASOTA) with rows of: 47076
HOUSTON with rows of: 192993
BALTIMORE with rows of: 115537
ORLANDO-DAYTONA BCH-MELBRN with rows of: 32449
MINNEAPOLIS-ST. PAUL with rows of: 149519
DETROIT with rows of: 132684
MIAMI-FT. LAUDERDALE with rows of: 77915
SAN DIEGO with rows of: 80396
SACRAMNTO-STKTON-MODESTO with rows of: 68785
AUSTIN with rows of: 62093
PORTLAND, OR with rows of: 78090
WEST PALM BEACH-FT. PIERCE with rows of: 40750
RALEIGH-DURHAM (FAYETVLLE) with rows of: 65922
CHARLOTTE with rows of: 51839
CLEVELAND-AKRON (CANTON) with rows of: 6503

In [20]:
writer.save()

In [21]:
gc.collect()

1255

# On Median Income Zip5

In [17]:
del EASI_1A
del DMA_Zips_29_High
del df

In [18]:
zip5_HH=DMA_Zips_29.groupby(['ZIP_CODE','DMA','state'])['HH18'].sum().to_frame().reset_index()

In [19]:
DMA_Zips_29.shape

(18382899, 7)

In [20]:
for col in DMA_Zips_29.columns.tolist():
    print(col, DMA_Zips_29[col].apply(lambda x: type(x)).unique().tolist())


ZIP_CODE [<class 'str'>]
DMA [<class 'str'>]
state [<class 'str'>]
ZIP4 [<class 'str'>, <class 'float'>]
HH18 [<class 'float'>]
Median_HH_Inc [<class 'float'>]
PCTHH200P [<class 'float'>]


In [21]:
temp_DMA_Zip_29=DMA_Zips_29[['ZIP_CODE','ZIP4','HH18','Median_HH_Inc','PCTHH200P']].drop_duplicates(['ZIP_CODE','ZIP4'])

In [22]:
del temp_DMA_Zip_29['ZIP4']
gc.collect()

181

In [23]:
i=0
zip5_HH_Median_Inc=pd.DataFrame()

for zip_cd,group in temp_DMA_Zip_29.groupby(['ZIP_CODE']):
    
    total_HH=group['HH18'].sum()
    group=group[['ZIP_CODE','HH18','Median_HH_Inc','PCTHH200P']]
    
    
    agg_Pctg_HH_200P=sum(group['PCTHH200P']*group['HH18']/total_HH)
    agg_median=sum(group['Median_HH_Inc']*group['HH18']/total_HH)
    
    df_app=pd.DataFrame({"ZIP_CODE":zip_cd,"Meidan_HH_Inc_agg":agg_median,"Pctg_HH_200P_agg":agg_Pctg_HH_200P},index=[zip_cd])
    
    zip5_HH_Median_Inc=zip5_HH_Median_Inc.append(df_app)
    
    i=i+1
    if i % 1000==10:
        print(i,datetime.datetime.now())
        logging.info(str(i)+"|"+str(datetime.datetime.now()))
        gc.collect()
    if i % 1000==100:
        print(i,datetime.datetime.now())
        logging.info(str(i)+"|"+str(datetime.datetime.now()))
    

10 2018-08-24 16:09:42.300079
100 2018-08-24 16:09:42.636527
1010 2018-08-24 16:09:45.862787
1100 2018-08-24 16:09:46.212065
2010 2018-08-24 16:09:49.537439
2100 2018-08-24 16:09:49.894996
3010 2018-08-24 16:09:53.148144
3100 2018-08-24 16:09:53.493987
4010 2018-08-24 16:09:56.784119
4100 2018-08-24 16:09:57.141208
5010 2018-08-24 16:10:00.562325
5100 2018-08-24 16:10:00.935650
6010 2018-08-24 16:10:04.485141
6100 2018-08-24 16:10:04.865111
7010 2018-08-24 16:10:08.462312
7100 2018-08-24 16:10:08.832733
8010 2018-08-24 16:10:12.417193
8100 2018-08-24 16:10:12.784190
9010 2018-08-24 16:10:16.381049
9100 2018-08-24 16:10:16.761749
10010 2018-08-24 16:10:20.444912
10100 2018-08-24 16:10:20.845072
11010 2018-08-24 16:10:24.541232
11100 2018-08-24 16:10:24.915325
12010 2018-08-24 16:10:28.718805
12100 2018-08-24 16:10:29.119057
13010 2018-08-24 16:10:32.914616
13100 2018-08-24 16:10:33.309005
14010 2018-08-24 16:10:37.104270
14100 2018-08-24 16:10:37.505894


In [24]:
zip5_HH_Median_Inc.shape

(14289, 3)

In [25]:
zip5_HH_Median_Inc.tail(4)

,Meidan_HH_Inc_agg,Pctg_HH_200P_agg,ZIP_CODE
99133,46810.562602,3.715728,99133
99322,49071.046543,3.479920,99322
99350,62290.829562,4.673914,99350
99356,49137.184874,3.549220,99356


In [26]:
# Zip 5 level HH Inc >= $ 100K
zip5_HH_Median_Inc_High=zip5_HH_Median_Inc[zip5_HH_Median_Inc['Meidan_HH_Inc_agg']>=100000]
zip5_HH_Median_Inc_High=pd.merge(zip5_HH_Median_Inc_High,zip5_HH,on="ZIP_CODE",how="left")
zip5_HH_Median_Inc_High['HH_200K_Plus']=zip5_HH_Median_Inc_High['HH18']*zip5_HH_Median_Inc_High['Pctg_HH_200P_agg']/100

In [27]:
unique_zip_city=pd.DataFrame()
for zip_cd in zip5_HH_Median_Inc_High['ZIP_CODE'].unique().tolist():
    if len(zipcodes.matching(zip_cd))>=1:
        city=zipcodes.matching(zip_cd)[0]['city']
        df=pd.DataFrame({"ZIP_CODE":zip_cd,"city":city},index=[zip_cd])
        unique_zip_city=unique_zip_city.append(df)
    else:
        print(zip_cd)
# No need to fill through google

In [28]:
zip5_HH_Median_Inc_High=pd.merge(zip5_HH_Median_Inc_High,unique_zip_city,on="ZIP_CODE",how="left")

In [29]:
zip5_HH_Median_Inc_High.head(2)

,Meidan_HH_Inc_agg,Pctg_HH_200P_agg,ZIP_CODE,DMA,state,HH18,HH_200K_Plus,city
0,108650.138276,10.797574,01068,BOSTON (MANCHESTER),MA,729.7,78.7899,OAKHAM
1,105144.696711,14.519488,01074,BOSTON (MANCHESTER),MA,246.3,35.7615,SOUTH BARRE


In [30]:
zip5_HH_Median_Inc_High=zip5_HH_Median_Inc_High[['ZIP_CODE','DMA','city','state','HH18','Meidan_HH_Inc_agg','Pctg_HH_200P_agg','HH_200K_Plus']]

In [31]:
writer=pd.ExcelWriter("/home/jian/Projects/Saatva/High_Income_Zips_for_29_DMAs/Saatva_Zip5 Median HH Inc above 100K for 29 DMAs_JL_20180823.xlsx",engine="xlsxwriter")
zip5_HH_Median_Inc_High.to_excel(writer,"all_29_DMAs",index=False)

for dma in DMA_29:
    df=zip5_HH_Median_Inc_High[zip5_HH_Median_Inc_High['DMA']==dma]
    df.to_excel(writer,dma,index=False)


In [32]:
writer.save()